In [56]:
import sys
import os

project_root = os.path.abspath("../..")

if project_root not in sys.path:
    sys.path.append(project_root)

In [57]:
from datapipeline.utils.spark_session import get_spark_session

spark = get_spark_session("Bronze_NewsDataHub")
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

In [60]:
from pyspark.sql import Row
from pyspark.sql.functions import xxhash64, col

newsdatahub_data_path = "../../sanewsstorage/main/newsdatahub"

# Read parquet
newsdatahub_df = (
    spark.read
         .option("recursiveFileLookup", "true")
         .parquet(newsdatahub_data_path)
)

# Rename columns with "."
for col_name in newsdatahub_df.columns:
    if "." in col_name:
        newsdatahub_df = newsdatahub_df.withColumnRenamed(
            col_name,
            col_name.replace(".", "_")
        )

newsdatahub_df = newsdatahub_df.withColumn(
    "hash",
    xxhash64(*[col(f"`{c}`") for c in newsdatahub_df.columns])
)

newsdatahub_df.printSchema()
newsdatahub_df.show(20)

# -------------------------------
# SAFE ROW FILTERING
# -------------------------------
def safe_row(row):
    try:
        # Try to convert to dict, which forces serialization
        _ = row.asDict()
        return True
    except:
        return False

# Apply safe_row using RDD
rdd = newsdatahub_df.rdd
safe_rdd = rdd.filter(safe_row)

safe_count = safe_rdd.count()
total_count = rdd.count()
dropped_count = total_count - safe_count

print("="*30)
print(f"Total rows   : {total_count}")
print(f"Written rows : {safe_count}")
print(f"Dropped rows : {dropped_count}")
print("="*30)

# Convert back to DataFrame
safe_df = spark.createDataFrame(safe_rdd, schema=newsdatahub_df.schema)


root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- source_title: string (nullable = true)
 |-- source_link: string (nullable = true)
 |-- article_link: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- topics: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: string (nullable = true)
 |-- pub_date: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- content: string (nullable = true)
 |-- media_url: string (nullable = true)
 |-- media_type: string (nullable = true)
 |-- media_description: string (nullable = true)
 |-- media_credit: string (nullable = true)
 |-- media_thumbnail: string (nullable = true)
 |-- language: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- source_country: string (nullable = true)
 |-- source_political_leaning: string (nullable = true)
 |-- source_reliability_score: double (nullable = true

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 5.0 failed 1 times, most recent failure: Lost task 7.0 in stage 5.0 (TID 19) (DESKTOP-IM373TG executor driver): org.apache.spark.SparkException: Parquet column cannot be converted in file file:///c:/Users/Echelon/Desktop/re/sa-news/sanewsstorage/main/newsdatahub/2026/02/10/09.parquet. Column: [media_description], Expected: string, Found: INT32.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:854)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:287)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:129)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:593)
	at org.apache.spark.sql.catalyst.ex

In [61]:
from delta.tables import DeltaTable
import os

bronze_path = "../../sanewsstorage/bronze/deltatables/newsdatahub_delta"

if DeltaTable.isDeltaTable(spark, bronze_path):

    delta_table = DeltaTable.forPath(spark, bronze_path)

    (
        delta_table.alias("t")
        .merge(
            newsdatahub_df.alias("s"),
            "t.hash = s.hash"
        )
        .whenNotMatchedInsertAll()
        .execute()
    )

else:
    (
        newsdatahub_df.write
          .format("delta")
          .mode("overwrite")
          .save(bronze_path)
    )

Py4JJavaError: An error occurred while calling o1353.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 6.0 failed 1 times, most recent failure: Lost task 6.0 in stage 6.0 (TID 26) (DESKTOP-IM373TG executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to file:/c:/Users/Echelon/Desktop/re/sa-news/sanewsstorage/bronze/deltatables/newsdatahub_delta.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:774)
	at org.apache.spark.sql.delta.files.DeltaFileFormatWriter$.executeTask(DeltaFileFormatWriter.scala:447)
	at org.apache.spark.sql.delta.files.DeltaFileFormatWriter$.$anonfun$executeWrite$2(DeltaFileFormatWriter.scala:274)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:93)
	at org.apache.spark.TaskContext.runTaskWithListeners(TaskContext.scala:161)
	at org.apache.spark.scheduler.Task.run(Task.scala:141)
	at org.apac

In [49]:
from delta.tables import DeltaTable

delta_table = DeltaTable.forPath(spark, bronze_path)
df = delta_table.toDF()
df.show(5)

files = delta_table.toDF().inputFiles
print(files)


AnalysisException: [DELTA_TABLE_NOT_FOUND] Delta table `../../sanewsstorage/bronze/deltatables/newsdatahub_delta` doesn't exist.